In [1]:
import numpy as np

<h4>Загрузка простого .csv файла с помощью Numpy</h4>

In [3]:
data = np.genfromtxt('data/les2_data_example.csv') # ('path/to/myfile.csv',delimiter=',',skiprows=0, dtype=float)
data

array([ 64.,  40., 107., ...,  59.,  65.,  62.])

<h4>Пример</h4>
Пусть N=30463.<br> 
Для заданной выборки рассчитаем среднее, медиану, дисперсию, стандартное отклонение, 1й и 3ю квартиль. <br>
Также рассчитаем 98.0%-ый доверительный интервал для <b>суммы</b> 

In [4]:
N = 30463
n = data.size # или len(data)
n

1015

Сначала рассчитаем оценку среднего значения выборки

In [5]:
y_summ = data.sum()
y_summ_2 = np.power(data,2).sum()
y_mean = y_summ/n
print(y_mean)
print(data.mean())

54.570443349753695
54.570443349753695


Далее рассчитаем оценку общего (суммарного значения) для генеральной совокупности

In [6]:
Y_summ = y_mean*N
Y_summ

1662379.4157635467

Рассчитаем долю отбора

In [7]:
f = n/N
f

0.033319108426615895

Рассчитаем выборочную дисперсию

In [8]:
var = np.sum(np.power(data,2) - np.power(y_mean,2)) / (n-1)
var

481.66539773224105

In [9]:
data.var(ddof=1)

481.6653977322412

In [11]:
#Будем использовать эту формулу
var = (y_summ_2 - np.power(y_summ,2)/n) / (n-1)
var

481.6653977322412

И стандартное отклонение

In [12]:
std = np.sqrt(var)
std

21.946876719302026

In [13]:
data.std(ddof=1)

21.946876719302026

Стандартная ошибка среднего (насколько оценка среднее значение отклоняется от среднего)

In [14]:
std_mean_error = (std * np.sqrt(1-f)) / np.sqrt(n)
std_mean_error

0.6773003031231355

Стандартная ошибка суммарного значения (насколько оценка суммарного значения отклоняется от суммы ГС)

In [15]:
std_summ_error = N * (std * np.sqrt(1-f)) / np.sqrt(n)
std_summ_error

20632.599134040076

Для того, чтобы рассчитать доверительные интервалы нужно найти квантиль t уровня &alpha; нормального распределения N(0,1). В приведенной формуле, &alpha; - вероятность ошибки. Если нам необходимы 80% доверительные границы, то &alpha; = 20%.

In [16]:
from scipy.stats import norm

In [17]:
alpha =1-0.98

In [17]:
#Левая квантиль
t_2a=norm.ppf(alpha/2)
t_2a

-2.3263478740408408

In [18]:
#Правая квантиль
t_1_2a=norm.ppf(1-alpha/2)
t_1_2a

2.3263478740408408

Рассчитаем левую и правую границу доверительного интервала для <b>среднего значения</b>

In [19]:
Y_mean_left =y_mean + t_2a * std_mean_error
Y_mean_right =y_mean + t_1_2a * std_mean_error
(Y_mean_left,Y_mean_right)

(52.99480722949597, 56.14607947001142)

Видим, что t_2a и t_1_2a симметричны, поэтому можно так:

In [18]:
t = norm.ppf(1-alpha/2)
Y_mean_left =y_mean - t * std_mean_error
Y_mean_right =y_mean + t * std_mean_error
(Y_mean_left,Y_mean_right)

(52.99480722949597, 56.14607947001142)

Рассчитаем левую и правую границу доверительного интервала для <b>суммы</b>

In [19]:
Y_mean_left * N, Y_mean_right * N

(1614380.8126321356, 1710378.0188949578)

<h4>Найдем квартили</h4>

In [20]:
data.sort()
data

array([  9.,  13.,  15., ..., 184., 202., 215.])

In [21]:
len(data)

1015

Количество элементов <b>нечетное</b>. Вычисляем индекс по формуле для n = 2k + 1

In [22]:
k=int((n-1)/2) # n-1 т.к. нумерация индексов начинается с 0
k

507

In [23]:
q2 = data[k]
q2

51.0

In [24]:
q1_data = data[:k] # [0:k)
q3_data = data[k+1:] # (k:n]

элемент под номером 508 (507й индекс) делит выборку на 2 ровных части по 507 элементов. Он не включается ни в одну из этих частей

In [26]:
len(q1_data), len(q3_data)

(507, 507)

Проделаем то же для оставшихся наборов

In [27]:
k1 = int((len(q1_data) - 1) / 2)
k3 = int((len(q3_data) - 1) / 2)

In [28]:
q1 = q1_data[k1]
q3 = q3_data[k3]

In [30]:
print(q1, q2, q3) # по формуле
print(np.median(q1_data), np.median(data), np.median(q3_data)) # сравним с методом median
print(np.percentile(data, 25), np.percentile(data, 50), np.percentile(data, 75)) # сравним с методом percentile

39.0 51.0 64.0
39.0 51.0 64.0
39.0 51.0 64.0


<h4>Сохраняем в .csv</h4>
Используем библиотеку <b>pandas</b>

In [31]:
import pandas as pd

Создаем словарь со всеми необходимыми значениями. round() для округления до 3х знаков после запятой (в лабе не выполняем)

In [32]:
d = {'mean': round(y_mean, 3),
     'median': round(q2, 3),
     'var': round(var, 3),
     'std_value': round(std, 3),
     'quantile1': round(q1, 3),
     'quantile3': round(q3, 3),
     'CI_left': round(Y_mean_left * N, 3),
     'CI_right': round(Y_mean_right * N, 3)
    }
d

{'mean': 54.57,
 'median': 51.0,
 'var': 481.665,
 'std_value': 21.947,
 'quantile1': 39.0,
 'quantile3': 64.0,
 'CI_left': 1614380.813,
 'CI_right': 1710378.019}

Создаем <b>DataFrame</b>

In [41]:
df = pd.DataFrame(index=[0],data=d)
df

,mean,median,var,std_value,quantile1,quantile3,CI_left,CI_right
0,54.57,51.0,481.665,21.947,39.0,64.0,1614380.813,1710378.019


Сохраняем в файл .csv

In [42]:
df.to_csv('data/result.csv', index=False)

Как прочитать .csv файл с помощью pandas

In [45]:
df1 = pd.read_csv('data/les2_data_example.csv')
df1

,col_name
0,64
1,40
2,107
3,74
4,34
...,...
1010,77
1011,51
1012,59
1013,65


In [49]:
df1['col_name'].values

481.1908505423573